In [ ]:
from glob import glob
import json
from tqdm import tqdm

In [ ]:
selected = [
    '/home/husein/ssd3/instructions/synthetic-codealpaca-v1-chatgpt4.jsonl',
    '/home/husein/ssd3/instructions/synthetic-glaive_coder_raw_text.jsonl',
    '/home/husein/ssd3/instructions/synthetic-oss_instruct-decontaminated.jsonl'
]

In [ ]:
rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'jawaban',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas',
    'pertama-tama',
    'bagian',
    'fungsionalitas',
]

indons = []
def found_word(s, words):
    global indons
    for i in range(len(words)):
        if words[i] in s:
            indons.append(s)
            return True
    return False

In [ ]:
def replace_code(s):
    s = s.replace(' Kode ', ' Kod ')
    s = s.replace(' kode ', ' kod ')
    s = s.replace('Kode ', 'Kod ')
    s = s.replace('kodenya', 'kodnya')
    s = s.replace('Kodenya', 'Kodnya')
    s = s.replace('opkode', 'opkod')
    s = s.replace(' kode', ' kod')
    s = s.replace('dikodekan', 'dikodkan')
    s = s.replace('kodks', 'kodeks')
    s = s.replace('kode ', 'kod ')
    s = s.replace(' Kode', ' Kod')
    return s

In [ ]:
english_news = glob('/home/husein/ssd3/google-translate-english-news/*.requested')
len(english_news)

In [ ]:
english_texts = glob('/home/husein/ssd3/google-translate-english-texts/*.requested')
len(english_texts)

In [ ]:
malay_news = glob('/home/husein/ssd3/google-translate-malay-news/*.requested')
len(malay_news)

In [ ]:
hansard = glob('/home/husein/ssd3/google-translate-malaysia-hansard/*.requested')
len(hansard)

In [ ]:
ens = [english_news, english_texts]
mss = [malay_news, hansard]

In [ ]:
def reject(k):
    if 'return JSON' in k:
        return
    if 'AI language model' in k:
        return
    if 'model bahasa AI' in k:
        return
    return True

In [ ]:
with open('train.json', 'w') as fopen_jsonl:
    
    for f in selected:
        with open(f) as fopen:
            for l in fopen:
                l = json.loads(l)
                en = l['instruction_en']
                ms = l['instruction_ms']

                if not en or not ms:
                    continue

                if not reject(en) or not reject(ms):
                    continue
                
                ms = replace_code(ms)
        
                if found_word(ms, rejected_words):
                    continue
                
                if len(en) and len(en.split()) < 1200 and len(ms) and len(ms.split()) < 1200:
                    d = {"translation": {"src": ms, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')

                    d = {"translation": {"src": en, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
                

In [ ]:
with open('train.json', 'a') as fopen_jsonl:
    with open('/home/husein/ssd3/translation/negeri-kelantan.jsonl') as fopen:
        for l in fopen:
            l = json.loads(l)
            left = l['r']['translation']
            right = l['original']['ms']
            
            if len(left) and len(left.split()) < 1200 and len(right) and len(right.split()) < 1200:

                d = {"translation": {"src": left, "tgt": right, 'prefix': 'terjemah ke Melayu: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
            
    with open('/home/husein/ssd3/translation/negeri-utara.jsonl') as fopen:
        for l in fopen:
            l = json.loads(l)
            left = l['r']['translation']
            right = l['original']['ms']
            
            if len(left) and len(left.split()) < 1200 and len(right) and len(right.split()) < 1200:

                d = {"translation": {"src": left, "tgt": right, 'prefix': 'terjemah ke Melayu: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')

In [ ]:
with open('train.json', 'a') as fopen_jsonl:
    with open('/home/husein/ssd3/gpt4all-1.3/translated-gpt4all-filtered-noncode.jsonl') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            try:
                en = l['src']['prompt']
                ms = l['r'][0]['result']
                
                if not reject(en) or not reject(ms):
                    continue
                    
                ms = replace_code(ms)
        
                if found_word(ms, rejected_words):
                    continue

                if len(en) and len(en.split()) < 1200 and len(ms) and len(ms.split()) < 1200:
                    d = {"translation": {"src": ms, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')

                    d = {"translation": {"src": en, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')

            except:
                pass
            
            try:
                en = l['src']['response']
                ms = l['r'][1]['result']
                
                if not reject(en) or not reject(ms):
                    continue
                    
                ms = replace_code(ms)
        
                if found_word(ms, rejected_words):
                    continue
                
                if len(en) and len(en.split()) < 1200 and len(ms) and len(ms.split()) < 1200:
                    d = {"translation": {"src": ms, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')

                    d = {"translation": {"src": en, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
            
            except:
                pass

In [ ]:
!wc -l train.json

In [ ]:
!ls -lh /home/husein/ssd3/translation/process-manglish.jsonl

In [ ]:
with open('train.json', 'a') as fopen_jsonl:
    with open('/home/husein/ssd3/translation/process-manglish.jsonl') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            left = l['left']
            en = l['en']
            ms = l['ms']
            
            if not reject(en) or not reject(ms):
                continue

            if len(left) and len(left.split()) < 1200 and len(en) and len(en.split()) < 1200:
                d = {"translation": {"src": left, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                
                d = {"translation": {"src": en, "tgt": left, 'prefix': 'terjemah ke Manglish: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                
            if len(left) and len(left.split()) < 1200 and len(ms) and len(ms.split()) < 1200:
                d = {"translation": {"src": left, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                
                d = {"translation": {"src": ms, "tgt": left, 'prefix': 'terjemah ke Manglish: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                
            if len(en) and len(en.split()) < 1200 and len(ms) and len(ms.split()) < 1200:
                d = {"translation": {"src": en, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                
                d = {"translation": {"src": ms, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')

In [ ]:
with open('train.json', 'a') as fopen_jsonl:
    for f in [
        '/home/husein/ssd3/translation/processed-ms-id.jsonl', 
        '/home/husein/ssd3/translation/processed-ms-jw.jsonl'
    ]:
        with open(f) as fopen:
            for l in tqdm(fopen):
                l = json.loads(l)
                left = l['left']
                right = l['right']
                if len(left) and len(left.split()) < 1200 and len(right) and len(right.split()) < 1200:
                    d = {"translation": {"src": right, "tgt": left, 'prefix': 'terjemah ke Melayu: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
                    
with open('train.json', 'a') as fopen_jsonl:
    with open('/home/husein/ssd3/translation/process-bjn-Latn.jsonl') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            left = l['left']
            chatgpt = l['chatgpt']
            google = l['google']
            
            if not reject(chatgpt) or not reject(google):
                continue
            
            if len(left) and len(left.split()) < 1200 and len(google) and len(google.split()) < 1200:
                d = {"translation": {"src": left, "tgt": google, 'prefix': 'terjemah ke Melayu: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')
                
            if len(chatgpt) and len(chatgpt.split()) < 1200 and len(google) and len(google.split()) < 1200:
                d = {"translation": {"src": chatgpt, "tgt": google, 'prefix': 'terjemah ke Melayu: '}}
                fopen_jsonl.write(f'{json.dumps(d)}\n')

In [ ]:
!wc -l train.json

In [ ]:
with open('train.json', 'a') as fopen_jsonl:
    for f in [
        '/home/husein/ssd3/translation/processed-twitter.jsonl', 
        '/home/husein/ssd3/translation/processed-facebook.jsonl', 
        '/home/husein/ssd3/translation/processed-iium-confession.jsonl',
        '/home/husein/ssd3/translation/processed-b.cari.com.my.jsonl',
    ]:
        with open(f) as fopen:
            for l in tqdm(fopen):
                try:
                    l = json.loads(l)
                    left = l['left']
                    en = l['en']
                    ms = l['ms']
                    
                    if not reject(en) or not reject(ms):
                        continue

                    if len(left) and len(left.split()) < 1200 and len(en) and len(en.split()) < 1200:
                        d = {"translation": {"src": left, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')

                        d = {"translation": {"src": en, "tgt": left, 'prefix': 'terjemah ke pasar Melayu: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')

                    if len(left) and len(left.split()) < 1200 and len(ms) and len(ms.split()) < 1200:
                        d = {"translation": {"src": left, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')

                        d = {"translation": {"src": ms, "tgt": left, 'prefix': 'terjemah ke pasar Melayu: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')

                    if len(ms) and len(ms.split()) < 1200 and len(en) and len(en.split()) < 1200:
                        d = {"translation": {"src": en, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')

                        d = {"translation": {"src": ms, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')
                        
                except Exception as e:
                    pass

In [ ]:
!wc -l train.json

In [ ]:
with open('train.json', 'a') as fopen_jsonl:
    with open('gpt4all-code.jsonl') as fopen:
        for l in tqdm(fopen):
            try:
                l = json.loads(l)
                left = l['prompt'][0]
                right = l['prompt'][1]
                
                right = replace_code(right)
        
                if found_word(right, rejected_words):
                    continue
                
                if len(left) and len(right):
                    d = {"translation": {"src": left, "tgt": right, 'prefix': 'terjemah ke Melayu: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
                    
                    d = {"translation": {"src": right, "tgt": left, 'prefix': 'terjemah ke Inggeris: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
                
                left = l['response'][0]
                right = l['response'][1]
                
                right = replace_code(right)
        
                if found_word(right, rejected_words):
                    continue
                
                if len(left) and len(right):
                    d = {"translation": {"src": left, "tgt": right, 'prefix': 'terjemah ke Melayu: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
                    
                    d = {"translation": {"src": right, "tgt": left, 'prefix': 'terjemah ke Inggeris: '}}
                    fopen_jsonl.write(f'{json.dumps(d)}\n')
            except Exception as e:
                print(e)

In [ ]:
!wc -l train.json

In [ ]:
import random

with open('train.json', 'a') as fopen_jsonl:
    for fs in ens:
        for f in fs:
            with open(f) as fopen:
                for l in tqdm(fopen):
                    
                    if random.random() > 0.4:
                        continue
                        
                    data = json.loads(l)
                    en = data['src']
                    ms = data['r']['result']

                    if len(en) and len(en.split()) < 1200 and len(ms) and len(ms.split()) < 1200:
                        d = {"translation": {"src": en, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')
                        
                        d = {"translation": {"src": ms, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')
                        
    for fs in mss:
        for f in fs:
            with open(f) as fopen:
                for l in tqdm(fopen):
                    
                    if 'malaysia-hansard' not in f and random.random() > 0.6:
                        continue
                    
                    data = json.loads(l)
                    if isinstance(data['src'], dict):
                        s = data['src']['cleaned']
                    else:
                        s = data['src']
                    ms = s
                    en = data['r']['result']

                    if len(en) and len(en.split()) < 1200 and len(ms) and len(ms.split()) < 1200:
                        
                        d = {"translation": {"src": en, "tgt": ms, 'prefix': 'terjemah ke Melayu: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')
                        
                        d = {"translation": {"src": ms, "tgt": en, 'prefix': 'terjemah ke Inggeris: '}}
                        fopen_jsonl.write(f'{json.dumps(d)}\n')

In [ ]:
!wc -l train.json

In [ ]:
def found_word(s, words):
    for i in range(len(words)):
        if words[i] in s:
            return words[i]
    return False

In [ ]:
!shuf train.json > shuffled-train.json

In [ ]:
!head -n 1 shuffled-train.json

In [ ]:
# !pip3.8 install mosaicml-streaming

In [ ]:
from streaming import MDSWriter, LocalDataset

columns = {
    'src': 'str',
    'tgt': 'str',
    'prefix': 'str',
}
hashes = 'sha1', 'xxh64'

In [ ]:
!rm -rf mosaic

In [ ]:
with MDSWriter(out='mosaic', columns=columns, compression=None, hashes=hashes) as out:
    with open('shuffled-train.json') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)['translation']
            out.write(l)

In [ ]:
dataset = LocalDataset('mosaic')

In [ ]:
len(dataset)

In [ ]:
len(indons)

In [ ]:
for i in indons:
    if 'pertama-tama' in i:
        print(i)
        break

In [ ]:
dataset[10]

In [3]:
rm -rf train.json shuffled-train.json

In [ ]:
!du -hs mosaic

In [ ]:
!cp mosaic/* ~/ssd3/mosaic-noisy-translation

In [ ]:
!ls -lh